In [1]:
!unzip /content/email.zip

Archive:  /content/email.zip
   creating: email/
  inflating: email/0067.1999-12-27.farmer.ham.txt  
  inflating: __MACOSX/email/._0067.1999-12-27.farmer.ham.txt  
  inflating: email/0272.2000-01-28.farmer.ham.txt  
  inflating: __MACOSX/email/._0272.2000-01-28.farmer.ham.txt  
  inflating: email/0004.1999-12-14.farmer.ham.txt  
  inflating: __MACOSX/email/._0004.1999-12-14.farmer.ham.txt  
  inflating: email/0256.2004-01-20.GP.spam.txt  
  inflating: __MACOSX/email/._0256.2004-01-20.GP.spam.txt  
  inflating: email/0301.2000-02-02.farmer.ham.txt  
  inflating: __MACOSX/email/._0301.2000-02-02.farmer.ham.txt  
  inflating: email/0328.2004-01-29.GP.spam.txt  
  inflating: __MACOSX/email/._0328.2004-01-29.GP.spam.txt  
  inflating: email/0283.2000-01-31.farmer.ham.txt  
  inflating: __MACOSX/email/._0283.2000-01-31.farmer.ham.txt  
  inflating: email/0335.2000-02-04.farmer.ham.txt  
  inflating: __MACOSX/email/._0335.2000-02-04.farmer.ham.txt  
  inflating: email/4185.2001-04-04.farmer.h

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
import os
import nltk
import time
import string
import operator
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

def text_cleanup(text):
    text_without_punctuation = [c for c in text if c not in string.punctuation]
    text_without_punctuation = ''.join(text_without_punctuation)
    text_without_stopwords = [word for word in text_without_punctuation.split() if word.lower() not in stopwords.words('english')]
    text_without_stopwords = ' '.join(text_without_stopwords)
    cleaned_text = [word.lower() for word in text_without_stopwords.split()]
    return cleaned_text


def prepare_dictionary():
  start_time = time()

  lmtzr = WordNetLemmatizer()
  k=0
  count = {}

  directory_in_str = "/content/email"
  directory = os.fsencode(directory_in_str)

  for file in os.listdir(directory):
      file = file.decode("utf-8")
      file_name = str(os.getcwd()) + '/email/'
      file_name = file_name + file
      print(file_name)
      file_reading = open(file_name,"r",encoding='utf-8', errors='ignore')
      words = text_cleanup(file_reading.read())
      for word in words:
          if (word.isdigit()==False and len(word)>2):
              word = lmtzr.lemmatize(word)
              if word in count:
                  count[word] += 1
              else:
                  count[word] = 1
      k+=1
      if(k%100==0):
          print("Done " + str(k))

  sorted_count = sorted(count.items(),key=operator.itemgetter(1),reverse=True)
  sorted_count = dict(sorted_count)

  f= open("wordslist.csv","w+")
  f.write('word,count')
  f.write('\n')
  for word , times in sorted_count.items():
      if times < 100:
          break
      f.write(str(word) + ',' + str(times))
      f.write('\n')
  f.close()

  print('Time (in seconds) to pre process the emails ' + str(round(time() - start_time,2)))

In [4]:
text_before = "Subject: vastar resources , inc ."
text_after = text_cleanup(text_before)
print("before cleanup: ", text_before)
print("after cleanup: ", text_after)

before cleanup:  Subject: vastar resources , inc .
after cleanup:  ['subject', 'vastar', 'resources', 'inc']


In [5]:
import os
import string
import numpy as np
import pandas as pd
from time import time
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


def prepare_data():
  start_time = time()

  df = pd.read_csv('wordslist.csv',header=0)
  words = df['word']

  lmtzr = WordNetLemmatizer()

  directory_in_str = "email/"
  directory = os.fsencode(directory_in_str)
  f = open("frequency.csv","w+")
  for i in words:
      f.write(str(i) + ',')
  f.write('output')
  f.write('\n')
  f.close()

  k=0
  for file in os.listdir(directory):
      file = file.decode("utf-8")
      file_name = str(os.getcwd()) + '/email/'
      for i in file:
          if(i!='b' and i!="'"):
              file_name = file_name + i
      k+=1
      file_reading = open(file_name,"r",encoding='utf-8', errors='ignore')
      words_list_array = np.zeros(words.size)
      for word in file_reading.read().split():
          word = lmtzr.lemmatize(word.lower())
          if(word in stopwords.words('english') or word in string.punctuation or len(word)<=2 or word.isdigit()==True):
              continue
          for i in range(words.size):
              if(words[i]==word):
                  words_list_array[i] = words_list_array[i]+1
                  break
      f = open("frequency.csv","a")
      for i in range(words.size):
          f.write(str(int(words_list_array[i])) + ',')

      print("file_name", file_name, file_name.find("spam"))
      print(words_list_array)
      if(file_name.find("spam") > 0):
          f.write("1")
      elif (file_name.find("spam") == -1):
          f.write("0")
      f.write('\n')
      f.close()
      if(k%100==0):
          print("Done " + str(k))

  print("Time (in seconds) to segregate entire dataset to form input vector " + str(round(time() - start_time,2)))

In [6]:
prepare_dictionary()
prepare_data()

/content/email/0322.2000-02-03.farmer.ham.txt
/content/email/0068.1999-12-27.farmer.ham.txt
/content/email/0301.2000-02-02.farmer.ham.txt
/content/email/0086.1999-12-29.farmer.ham.txt
/content/email/0062.2003-12-22.GP.spam.txt
/content/email/0320.2000-02-03.farmer.ham.txt
/content/email/0314.2000-02-02.farmer.ham.txt
/content/email/0028.1999-12-17.farmer.ham.txt
/content/email/0004.1999-12-14.farmer.ham.txt
/content/email/0042.1999-12-21.farmer.ham.txt
/content/email/4183.2001-04-04.farmer.ham.txt
/content/email/0328.2004-01-29.GP.spam.txt
/content/email/0338.2000-02-04.farmer.ham.txt
/content/email/0009.1999-12-14.farmer.ham.txt
/content/email/0325.2004-01-27.GP.spam.txt
/content/email/0262.2000-01-26.farmer.ham.txt
/content/email/0083.1999-12-28.farmer.ham.txt
/content/email/0304.2000-02-02.farmer.ham.txt
/content/email/3553.2001-02-08.farmer.ham.txt
/content/email/0007.1999-12-14.farmer.ham.txt
/content/email/0025.1999-12-16.farmer.ham.txt
/content/email/0319.2000-02-03.farmer.ham.t

In [7]:
import pandas as pd
df1 = pd.read_csv('wordslist.csv')
df2 = pd.read_csv('frequency.csv',header=0)

input_output = df2.values
X = input_output[:,:-1]
y= input_output[:,-1:]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train,  y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
from sklearn.svm import SVC
import numpy as np

classifier = SVC(C = 100, kernel="linear")
classifier.fit(X_train,np.ravel(y_train))

SVC(C=100, kernel='linear')

In [9]:
df = pd.read_csv('wordslist.csv',header=0)
words = df['word']
lmtzr = WordNetLemmatizer()
def encode_email(fileName, words):
  file_reading = open(fileName,"r",encoding='utf-8', errors='ignore')
  words_list_array = np.zeros(words.size)
  for word in file_reading.read().split():
      word = lmtzr.lemmatize(word.lower())
      if(word in stopwords.words('english') or word in string.punctuation or len(word)<=2 or word.isdigit()==True):
          continue
      for i in range(words.size):
          if(words[i]==word):
              words_list_array[i] = words_list_array[i]+1
              break
  return words_list_array

In [10]:
normal_sample = encode_email('/content/email/0028.1999-12-17.farmer.ham.txt', words)
print(normal_sample)

spam_sample = encode_email('/content/email/0334.2004-01-30.GP.spam.txt', words)
print(spam_sample)

[0. 1. 0. 3. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 1.]


In [11]:
def check_email(fileName, words, classifier):
  email = encode_email(fileName, words)
  if(classifier.predict([email])[0] == 1):
    print("Spam email")
  else:
    print("Normal email")

In [12]:
result1 = check_email('/content/email/0028.1999-12-17.farmer.ham.txt', words, classifier)
result2 = check_email('/content/email/0334.2004-01-30.GP.spam.txt', words, classifier)


Normal email
Spam email
